In [1]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from catboost import CatBoostClassifier, Pool

### "F35" GroupBy mean 변환

In [2]:
''' Data Load '''
train, test = load_data()

''' Seed '''
seed_everything(42)

''' Data Load '''
train = preprocessing(train)
test = preprocessing(test, is_test = True)

''' Train / Vaild Split'''
param = {'max_depth': 1,
 'num_leaves': 306,
 'subsample': 0.9757365622458185,
 'subsample_freq': 8,
 'min_child_samples': 136}
X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]

''' Model '''
model = lgb.LGBMClassifier(random_state=42, **param)

''' K-Fold '''
valid_array, test_array, voting = Kfold(model, 2, X_train, y_train, test, True)

---------------- Start MissingValue ----------------


100%|██████████| 26/26 [00:33<00:00,  1.30s/it]


---------------- Change Dtype ----------------
Memory usage of dataframe is 11785.06 MB
Memory usage after optimization is: 7392.94 MB
Decreased by 37.3%
---------------- Start MissingValue ----------------


100%|██████████| 26/26 [00:04<00:00,  5.26it/s]


---------------- Change Dtype ----------------
Memory usage of dataframe is 1835.20 MB
Memory usage after optimization is: 1159.98 MB
Decreased by 36.8%
---------------- 1 fold의 Acc: 0.7100041781337538 ----------------
[0. 0. 0. ... 0. 0. 0.]
[0.26313517 0.18179171 0.23395397 ... 0.15149553 0.294972   0.29414395]


KeyboardInterrupt: 